<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/Jeel/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [1]:
!git clone 'https://github.com/sumandutta8877/Data_DA.git'    # Run this Line for only one time

Cloning into 'Data_DA'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [2]:
import pandas as pd
df_train = pd.read_csv('/content/Data_DA/Data/Train_data.csv')
df_test = pd.read_csv('/content/Data_DA/Data/Test_data.csv')
df_sample = pd.read_csv('/content/Data_DA/Data/Sample_submission.csv')

df1 = df_train            # df1 is the training dataframe
df2 = df_test             # df2 is the test dataframe

# Start Processing

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [4]:
labels = dict(zip([1,2,3,4,5],df1['popularity'].unique()))
df1.loc[(df1['popularity']=='very low'), 'popularity level'] = 1
df1.loc[(df1['popularity']=='low'), 'popularity level'] = 2
df1.loc[(df1['popularity']=='average'), 'popularity level'] = 3
df1.loc[(df1['popularity']=='high'), 'popularity level'] = 4
df1.loc[(df1['popularity']=='very high'), 'popularity level'] = 5

**Pre Processing**

In [5]:
import datetime as dt
import time as tm
today = dt.date.today()

# On train data set
df1['loudness'] = (-1)*df1['loudness']

df1['mode'] = df1['mode'].apply(lambda x: 1 if x == 'Major' else 0)
df1['explicit'] = df1['explicit'].apply(lambda x: 1 if x == 'Yes' else 0)

df1['release_date'] = pd.to_datetime(df1['release_date'])
df1['Today']=today
df1['Today']=pd.to_datetime(df1['Today'])
df1['days older']= (df1['Today']-df1['release_date']).dt.days



# On test data set
df2['loudness'] = (-1)*df2['loudness']

df2['mode'] = df2['mode'].apply(lambda x: 1 if x == 'Major' else 0)
df2['explicit'] = df2['explicit'].apply(lambda x: 1 if x == 'Yes' else 0)

df2['release_date'] = pd.to_datetime(df2['release_date'])
df2['Today']=today
df2['Today']=pd.to_datetime(df2['Today'])
df2['days older']= (df2['Today']-df2['release_date']).dt.days


In [6]:
df1.head()

,id,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,release_date,speechiness,tempo,valence,year,duration-min,popularity,popularity level,Today,days older
0,2015,0.949,0.235,0.0276,0,0.9270,5,0.513,27.398,1,1947-01-01,0.0381,110.838,0.0398,1947,3.0,very low,1.0,2021-03-14,27101
1,15901,0.855,0.456,0.4850,0,0.0884,4,0.151,10.046,1,2020-11-13,0.0437,152.066,0.8590,2020,2.4,low,2.0,2021-03-14,121
2,9002,0.827,0.495,0.4990,0,0.0000,0,0.401,8.009,0,1950-01-01,0.0474,108.004,0.7090,1950,2.6,very low,1.0,2021-03-14,26005
3,6734,0.654,0.643,0.4690,0,0.1080,7,0.218,15.917,1,1974-04-30,0.0368,83.636,0.9640,1974,2.4,low,2.0,2021-03-14,17120
4,15563,0.738,0.705,0.3110,0,0.0000,5,0.322,12.344,1,1973-01-01,0.0488,117.260,0.7850,1973,3.4,average,3.0,2021-03-14,17604


In [7]:
df1.columns

Index(['id', 'acousticness', 'danceability', 'energy', 'explicit',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'release_date', 'speechiness', 'tempo', 'valence', 'year',
       'duration-min', 'popularity', 'popularity level', 'Today',
       'days older'],
      dtype='object')

In [8]:
dfx = df1.drop(['id','release_date','popularity','popularity level', 'Today'],axis=1)
dfx.head()

,acousticness,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,year,duration-min,days older
0,0.949,0.235,0.0276,0,0.9270,5,0.513,27.398,1,0.0381,110.838,0.0398,1947,3.0,27101
1,0.855,0.456,0.4850,0,0.0884,4,0.151,10.046,1,0.0437,152.066,0.8590,2020,2.4,121
2,0.827,0.495,0.4990,0,0.0000,0,0.401,8.009,0,0.0474,108.004,0.7090,1950,2.6,26005
3,0.654,0.643,0.4690,0,0.1080,7,0.218,15.917,1,0.0368,83.636,0.9640,1974,2.4,17120
4,0.738,0.705,0.3110,0,0.0000,5,0.322,12.344,1,0.0488,117.260,0.7850,1973,3.4,17604


In [9]:
#features1 = ['days older', 'danceability', 'valence','acousticness','energy','key', 'liveness', 'loudness', 'speechiness','year']
# list_extra = ['acousticness','energy','key', 'liveness', 'loudness', 'speechiness']

X = dfx.values
y = df1['popularity level'].values

#to normalize the data
scaler = MinMaxScaler()
X1 = scaler.fit_transform(X)

In [10]:
type(X),type(y),X.shape,y.shape

(numpy.ndarray, numpy.ndarray, (12227, 15), (12227,))

In [11]:
X1

array([[0.9528112 , 0.23979592, 0.02758026, ..., 0.26732673, 0.03856749,
        0.73485684],
       [0.85843359, 0.46530612, 0.48498955, ..., 0.99009901, 0.03030303,
        0.00949052],
       [0.83032111, 0.50510204, 0.49898983, ..., 0.2970297 , 0.03305785,
        0.70539051],
       ...,
       [0.32931657, 0.5622449 , 0.56399115, ..., 0.63366337, 0.03168044,
        0.37152843],
       [0.12248904, 0.06204082, 0.93899876, ..., 0.96039604, 0.0399449 ,
        0.04091948],
       [0.03815161, 0.39693878, 0.76799529, ..., 0.99009901, 0.0399449 ,
        0.01250168]])

In [ ]:
#cmap = cm.get_cmap('gnuplot')
#scatter = pd.plotting.scatter_matrix(X_train,c=y_train,s=40,hist_kwds={'bins':15},figsize=(12,12),cmap=cmap)

In [ ]:
#fig = plt.figure(figsize=(10,10))
#ax = fig.add_subplot(111,projection = '3d')
#ax.scatter(X_train['days older'],X_train['danceability'],X_train['valence'], c=y_train,s=3)
#ax.set_xlabel('days older')
#ax.set_ylabel('danceability')
#ax.set_zlabel('valence')
#plt.show()


# **RandomForest classifier**

In [12]:
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

classifier = ensemble.RandomForestClassifier(n_jobs=-1)
param_grid = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": [1, 3, 5, 7],
    "criterion":['gini','entropy'],
}
model = model_selection.GridSearchCV(
    estimator = classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=10,
    n_jobs=1,
    cv=5,
)
model.fit(X1,y)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=1, n_estimators=100, score=0.465, total=   1.3s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV]  criterion=gini, max_depth=1, n_estimators=100, score=0.468, total=   0.4s
[CV] criterion=gini, max_depth=1, n_estimators=100 ...................


...

[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.474, total=   0.7s
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    6.0s remaining:    0.0s


[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.473, total=   0.8s
[CV] criterion=gini, max_depth=1, n_estimators=300 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=300, score=0.473, total=   1.0s
[CV] criterion=gini, max_depth=1, n_estimators=300 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=300, score=0.471, total=   1.1s
[CV] criterion=gini, max_depth=1, n_estimators=300 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=300, score=0.483, total=   1.1s
[CV] criterion=gini, max_depth=1, n_estimators=300 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=300, score=0.465, total=   1.0s
[CV] criterion=gini, max_depth=1, n_estimators=300 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=300, score=0.466, total=   1.0s
[CV] criterion=gini, max_depth=1, n_estimators=400 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=400, score=0.467, total=   1.4s
[CV] criterion

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  5.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              random_

In [13]:
results=pd.DataFrame(model.cv_results_)
#results.columns
results[['param_n_estimators','param_criterion','mean_test_score']]


,param_n_estimators,param_criterion,mean_test_score
0,100,gini,0.463237
1,200,gini,0.472153
2,300,gini,0.471906
3,400,gini,0.471253
4,100,gini,0.569888
5,200,gini,0.572177
6,300,gini,0.572831
7,400,gini,0.574630
8,100,gini,0.609717
9,200,gini,0.610208


In [ ]:
#dir(model)

In [14]:
model.best_score_,model.best_estimator_,model.best_params_

(0.6241932490255782,
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=7, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=200,
                        n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                        warm_start=False),
 {'criterion': 'gini', 'max_depth': 7, 'n_estimators': 200})

In [ ]:
#to normalize the data
#scaler = MinMaxScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# **Kernelized SVM  with diff kernels,gamma parameters**

In [ ]:
# Build an SVC (Support Vector Classification) model using linear regression
from sklearn import svm
linear = svm.SVC(kernel='linear', C=15).fit(X_train_scaled, y_train)
poly = svm.SVC(kernel='poly', degree=4, C=15).fit(X_train_scaled, y_train)
rbf = svm.SVC(kernel='rbf', gamma=1, C=15).fit(X_train_scaled, y_train)

print('Accuracy (linear Kernel) on test dataset: ', "%.2f" % (linear.score(X_test_scaled,y_test)*100))
print('Accuracy (linear Kernel) on train dataset: ', "%.2f" % (linear.score(X_train_scaled,y_train)*100))

print('\nAccuracy (Polynomial Kernel) on test dataset: ', "%.2f" % (poly.score(X_test_scaled,y_test)*100))
print('Accuracy (Polynomial Kernel) on train dataset: ', "%.2f" % (poly.score(X_train_scaled,y_train)*100))

print('\nAccuracy (RBF Kernel) on test dataset: ', "%.2f" % (rbf.score(X_test_scaled,y_test)*100))
print('Accuracy (RBF Kernel) on train dataset: ', "%.2f" % (rbf.score(X_train_scaled,y_train)*100))
